In [1]:
import torch
from functions import get_loader, get_model
from accelerate import Accelerator
import datetime

_, _, loader = get_loader()
model, optimizer, scheduler = get_model()

#使用4步梯度累积,梯度累计和mixed_precision不共存,不知道是否是一个bug
# accelerator = Accelerator(gradient_accumulation_steps=4)

#可以修改这里的mixed_precision,来查看不同精度的时间差,显存差
#no,fp8,fp16,bf16
accelerator = Accelerator(gradient_accumulation_steps=1,
                          mixed_precision='fp16')

loader, model, optimizer, scheduler = accelerator.prepare(
    loader, model, optimizer, scheduler)

now = datetime.datetime.now()
for i, data in enumerate(loader):
    #在这个范围内累积梯度
    with accelerator.accumulate(model):
        out = model(**data)
        accelerator.backward(out.loss)
        accelerator.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    if i % 1 == 0:
        lr = optimizer.state_dict()['param_groups'][0]['lr']

        labels = data['labels']
        logits = out['logits'].argmax(1)
        acc = (labels == logits).sum().item() / len(labels)

        print(i, len(loader), out.loss.item(), lr, acc)

datetime.datetime.now() - now

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


0 62 2.29443359375 9.990133642141359e-05 0.15625
1 62 1.952362060546875 9.96057350657239e-05 0.34375
2 62 1.7434158325195312 9.911436253643445e-05 0.4375
3 62 1.5019912719726562 9.842915805643155e-05 0.46875
4 62 1.1470603942871094 9.755282581475769e-05 0.5625
5 62 0.8336296081542969 9.648882429441257e-05 0.78125
6 62 0.4786376953125 9.524135262330098e-05 1.0
7 62 0.32486724853515625 9.381533400219318e-05 1.0
8 62 0.354766845703125 9.221639627510076e-05 0.96875
9 62 0.19248199462890625 9.045084971874738e-05 1.0
10 62 0.11542224884033203 8.852566213878947e-05 1.0
11 62 0.08633613586425781 8.644843137107059e-05 1.0
12 62 0.23552846908569336 8.422735529643444e-05 0.90625
13 62 0.1086416244506836 8.18711994874345e-05 0.96875
14 62 0.07680702209472656 7.938926261462366e-05 1.0
15 62 0.061490535736083984 7.679133974894983e-05 1.0
16 62 0.07937812805175781 7.408768370508576e-05 1.0
17 62 0.11747360229492188 7.128896457825364e-05 1.0
18 62 0.048834800720214844 6.840622763423391e-05 1.0
19 62 0

datetime.timedelta(seconds=16, microseconds=699176)

In [2]:
from accelerate import notebook_launcher


def f():
    print('f runed')


#在jupyter中也可以这样运行,主要就是可以增加一些参数
notebook_launcher(f, num_processes=0, mixed_precision='fp16')

Launching training on one GPU.
f runed
